In [1]:
import constant
import requests
from bs4 import BeautifulSoup as bs
import lxml
import json
import constant
import ast
import re

In [2]:
def get_html(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'}
    r = requests.get(url, headers = headers)
    return r

In [19]:
 r = get_html('https://www.fiesta.ru/spb/novelty/events/')

In [4]:
def fiesta_data_format(date):
    symbol = [',', '–']
    try:
        symbol_sep = [s for s in symbol if s in date]
        if symbol_sep:
            date_array = date.split(symbol_sep[0])
            date_start = date_array[0].strip()
            date_stop = date_array[-1].strip()
        else:
            date_start = date
            date_stop = None
    except:
        date_start = None
        date_stop = None
    return date_start, date_stop

In [ ]:
id_parse, type_event, img, title, date_start, date_stop, cost, discounted, address, metro, link

In [28]:
html = bs(r.text, 'html.parser')
event = []
soup = html.find_all('div', class_="grid_i grid_i__desktop-grid-1-3 grid_i__tablet-grid-1-2 grid_i__phone-grid-1-1")



for s in soup:
    id_parse = s.find('footer').attrs['data-calendar-item']
    type_event = 'fiesta' # !!!!!!!!!!
    #img = 'https://www.fiesta.ru' + s.find('img').get('src')
    title = s.find('a', class_='unit_t_a double-hover').get_text()
    # получаем дату события
    date = s.find('p', class_='unit_date').get_text()
    date_start, date_stop = fiesta_data_format(date)
    link = 'https://www.fiesta.ru' + s.find('a', class_='unit_t_a double-hover').get('href')
     
    # получение цены
    cost_html_link = get_html(link)
    cost_html_event = bs(cost_html_link.text, 'html.parser')
    cost = cost_html_event.find('div', class_='article_details').find_all('dd', class_='grid_i grid_i__desktop-grid-5-6 grid_i__tablet-grid-5-6 grid_i__phone-grid-1-1')[-1].get_text().strip()
    # если чтоимости нет -> то в cost попадает дата события, необходимо проверить ее наличие
    if any(x in cost for x in constant.date_format_kudago):
        cost = None 

    discounted = '0'
    try:
        full_address = s.find('p', class_='unit_place').get_text()
        if full_address.startswith('м.'):
            sep_full_address = full_address.split(','),
            address = ','.join(sep_full_address[1:]).strip()
            metro = sep_full_address[0]
        else:
            address = full_address
            metro = None
    except:
        address = None
        metro = None

    img = 'https://www.fiesta.ru'# + img_link

  
    print(id_parse, type_event, img, title, date_start, date_stop, address, metro, link , '\n\n', sep = '\n')
    #print(cost, title, link, img, date,  '\n', sep='\n')



46931
fiesta
https://www.fiesta.ru
Выставка «Лица и Лики»
26 марта
30 мая
Потемкинская ул., 4, «Ленинград Центр»
м. Чернышевская
https://www.fiesta.ru/spb/events/vystavka-litsa-i-liki/



46929
fiesta
https://www.fiesta.ru
Концерт Джанго
16 апреля
None
Садовая ул., 62, Hard Rock Cafe SPb
м. Садовая
https://www.fiesta.ru/spb/events/kontsert-dzhango-1618506973/



46927
fiesta
https://www.fiesta.ru
Планы на выходные: 15 самых интересных событий
16 апреля
18 апреля
None
None
https://www.fiesta.ru/spb/events/plany-na-vyhodnye-15-samyh-interesnyh-sobytiy-1618500845/



30088
fiesta
https://www.fiesta.ru
Вечеринки романтических знакомств от Dating-City 
16 апреля
25 апреля
None
None
https://www.fiesta.ru/spb/events/vecherinki-romanticheskih-znakomstv-ot-dating-city/



46912
fiesta
https://www.fiesta.ru
Открытие навигации на Неве
16 апреля
None
наб. Макарова, 34
None
https://www.fiesta.ru/spb/events/otkrytie-navigatsii-na-neve/



46921
fiesta
https://www.fiesta.ru
Концерт Smoke on the Water

In [6]:
id_parse = s.find('footer').attrs['data-calendar-item']
print(id_parse)

46855


In [116]:
full_address = 'м. Василеостровская, Кожевенная линия, 40, 2-й этаж, пространство «Севкабель Порт»; hgfchchf'
if full_address.find(';') != -1:
    full_address = full_address.split(';')[0]
if full_address.startswith('м.'):
    sep_full_address = full_address.split(',')
    address = ','.join(sep_full_address[1:]).strip()
    metro = sep_full_address[0]
    print(address)
    print(metro)

Кожевенная линия, 40, 2-й этаж, пространство «Севкабель Порт»
м. Василеостровская


In [63]:
def fff(address):
    metro = None
    result = re.findall(r'(м\.|метро|ст\.\s*метро|ст\.\s*м\.|ст\.\s*м)(\s*\S)', address, flags = re.IGNORECASE)
    
    if result:
        address = address.replace(''.join(result[0]), '')
        metro_re = result[0][1]
        metro_split = s.split('.' or ' ')
        metro = [x for x in constant.list_metro if re.findall(metro_split[-1] , x)]
    
    return address, metro

full_address = 'БКЗ Октябрьский, Лиговский пр., д.6. ст.м. "Пл. Восстания"'

print(fff(full_address))

('БКЗ Октябрьский, Лиговский пр., д.6. Пл. Восстания"', [])


In [91]:
s = 'БКЗ Октябрьский, Лиговский пр., д.6. ст.м. &quot;Площадь Ленина&quot;'
def sep_address_metro(address):
    metro = None
    result = re.search(r'м\.|метро|ст\.\s*метро|ст\.\s*м\.|ст\.\s*м', address, flags = re.IGNORECASE)
    if result:
        start_symbol = result.span()[0]
        end_symbol = result.span()[1]
   
        metro = address[end_symbol:].replace('\"', '').strip()
        metro = f'{metro}'
        address = address[0:start_symbol].strip()
      
    return address, metro


print(sep_address_metro(s))



('БКЗ Октябрьский, Лиговский пр., д.6.', '&quot;Площадь Ленина&quot;')


In [1]:
x = {"id": 435001186, "is_bot": false, "first_name": "Олег", "language_code": "ru"}

NameError: name 'false' is not defined